In [4]:
import pandas as pd
import os, sys
import numpy as np
import yaml
from catboost import CatBoostRegressor, Pool
sys.path.append("../steps/")
os.getcwd()

'c:\\Users\\LEI00010\\Documents\\progetti_personali\\deployactuarialprod\\notebooks'

In [11]:
from predict import Predictor
from ingest import Ingestion

In [9]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

In [28]:
cfg

{'data': {'train_path': 'data/train.csv',
  'test_path': 'data/test.csv',
  'valid_path': 'data/valid.csv'},
 'models': {'frequency': 'models/frequency_model.cbm',
  'severity': 'models/severity_model.cbm'},
 'mlflow': {'experiment_name': 'InsuranceApp',
  'run_name': 'frequency_severity',
  'tracking_uri': 'http://localhost:5000'}}

In [16]:
test_freq = pd.read_csv("../data/test.csv", sep=";", dtype_backend='pyarrow')
test_freq.head()

,VehPower,VehAge,DrivAge,Density,BonusMalus,VehBrand,VehGas,Region,Area,Exposure,ClaimNb,IDpol,claims_cost
0,7,0,46,76,50,B12,Diesel,R72,B,0.09,0,10,0.0
1,7,0,46,76,50,B12,Diesel,R72,B,0.84,0,11,0.0
2,7,0,33,137,68,B12,Diesel,R91,C,0.27,0,17,0.0
3,7,0,41,60,50,B12,Diesel,R52,B,0.75,0,25,0.0
4,9,0,44,27000,76,B12,Regular,R11,F,0.34,0,36,0.0


In [32]:
predictors = ['VehBrand', 'VehGas', 'Region','Area'] + ['VehPower', 'VehAge', 'DrivAge', 'Density', 'BonusMalus']
print(predictors)
example_dict = test_freq.loc[0, predictors].to_dict()

['VehBrand', 'VehGas', 'Region', 'Area', 'VehPower', 'VehAge', 'DrivAge', 'Density', 'BonusMalus']


In [33]:
example_dict

{'VehBrand': 'B12',
 'VehGas': 'Diesel',
 'Region': 'R72',
 'Area': 'B',
 'VehPower': 7,
 'VehAge': 0,
 'DrivAge': 46,
 'Density': 76,
 'BonusMalus': 50}

In [30]:
model_freq = CatBoostRegressor().load_model("../models/frequency_model.cbm")
model_sev: CatBoostRegressor = CatBoostRegressor().load_model("../models/severity_model.cbm")

In [36]:
predictor_freq = Predictor(model_freq, model_type="frequency")
predictor_sev = Predictor(model_sev, model_type="severity")

In [37]:
predictor_sev.predict(example_dict)

array([2360.61988938])

In [38]:
predictor_freq.predict(example_dict)

array([0.05529986])

In [39]:
predictor_freq.predict(test_freq)

KeyError: 'log_exposure'